In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [2]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
#from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3
from shapely.ops import transform, unary_union

In [4]:
import ee
# ee.Authenticate()

In [5]:
ee.Initialize()

In [6]:
OUTPUT_FILENAME = 'AQ-3-PM25exposure.csv'

In [7]:
THRESHOLD = 5.0

In [8]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"

In [9]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir +'/data/boundaries/boundary_georef.csv')
boundary_georef

,Unnamed: 0,geo_name,level,aoi_boundary_name,units_boundary_name,city_name,country_name,country_code,continent,city_id,project_name,city_boundary_name,geo_level
0,1,ARG-Mendoza,region,ADM3union,ADM3,Mendoza,Argentina,ARG,America,ARG-Mendoza_ADM-3-union_1,urbanshift,NaN,NaN
1,2,ARG-Mar_del_Plata,city,ADM3,ADM4,Mar del Plata city,Argentina,ARG,America,ARG-Mar_del_Plata_ADM-3_1,urbanshift,NaN,NaN
2,3,ARG-Ushuaia,city,ADM4,ADM5,Ushuaia city,Argentina,ARG,America,ARG-Ushuaia_ADM-4_1,urbanshift,NaN,NaN
3,4,ARG-Salta,region,ADM2union,ADM3,Salta,Argentina,ARG,America,ARG-Salta_ADM-2-union_1,urbanshift,NaN,NaN
4,5,ARG-Buenos_Aires,region,ADM2union,ADM2,Buenos Aires,Argentina,ARG,America,ARG-Buenos_Aires_ADM-2-union_1,urbanshift,NaN,NaN
5,6,BRA-Teresina,city,ADM4union,ADM4,Teresina city,Brazil,BRA,America,BRA-Teresina_ADM-4-union_1,urbanshift,NaN,NaN
6,7,BRA-Florianopolis,city,ADM4union,ADM4,Florianopolis,Brazil,BRA,America,BRA-Florianopolis_ADM-4-union_1,urbanshift,NaN,NaN
7,8,BRA-Belem,city,ADM4union,ADM4,Belem city,Brazil,BRA,America,BRA-Belem_ADM-4-union_1,urbanshift,NaN,NaN
8,9,CRI-San_Jose,region,ADM2union,ADM2,San Jose,Costa Rica,CRI,America,CRI-San_Jose_ADM-2-union_1,urbanshift,NaN,NaN
9,10,RWA-Kigali,city,ADM4union,ADM4,Kigali,Rwanda,RWA,Africa,RWA-Kigali_ADM-4-union_1,urbanshift,NaN,NaN


In [10]:
pm25_2020 = ee.Image("projects/wri-datalab/Cities4Forests/acag_pm25-surface")
worldpop_2020 = ee.ImageCollection("WorldPop/GP/100m/pop").filter(ee.Filter.eq('year', 2020)).mosaic().mask(pm25_2020)

In [11]:
exceedance_2020 = pm25_2020.gte(THRESHOLD)

In [12]:
threshold_percentage = pm25_2020.multiply(1.0/THRESHOLD).multiply(100.0)

In [13]:
exceedance_pop_img = worldpop_2020.mask(exceedance_2020)

In [14]:
def get_exceedancepops(fc):
    totalpop = worldpop_2020.reduceRegions(**{
        'reducer': ee.Reducer.sum(),
        'collection': fc,
        'scale': 10
    })
    exceedancepop = exceedance_pop_img.reduceRegions(**{
        'reducer': ee.Reducer.sum(),
        'collection': fc,
        'scale': 10
    })
    res = geemap.ee_to_pandas(exceedancepop)['sum'] / geemap.ee_to_pandas(totalpop)['sum']
    return res

def get_meanconc(fc):
    meanconc = threshold_percentage.reduceRegions(**{
        'reducer': ee.Reducer.mean(),
        'collection': fc,
        'scale': 10
    })
    res = geemap.ee_to_pandas(meanconc)['mean']
    return res

#    return threshold_percentage.reduceRegion(ee.Reducer.mean(), geom, 100, 'epsg:4326').getInfo()['b1']

In [15]:
results = []
for i in range(len(boundary_georef)):
    geo_name = boundary_georef.loc[i, 'geo_name']
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']
    for boundary_id in [boundary_id_aoi, boundary_id_unit]:
        if boundary_id[-3:] != 'nan':
            print("\n geo_name: " + boundary_id)
            # read boundaries
            boundary_path = aws_s3_dir + '/data/boundaries/boundary-'+ boundary_id + '.geojson'
            boundary_geo = requests.get(boundary_path).json()
            temp_gdf = gpd.GeoDataFrame.from_features(boundary_geo)
                        
            boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
            temp_gdf['exceedance_popfraction_2020'] = get_exceedancepops(boundary_geo_ee).fillna(-9999)
            temp_gdf['AQ_3_percentWHOlimitPM25exposure2020'] = get_meanconc(boundary_geo_ee).fillna(-9999)
            
            results.append(temp_gdf)
            gre_2_3 = pd.concat(results, axis=0)[['geo_id', 'geo_level', 'geo_name', 'geo_parent_name', 'exceedance_popfraction_2020', 'percent_of_WHO_2020']]
            gre_2_3.to_csv(OUTPUT_FILENAME)
print('Done')


 geo_name: ARG-Mendoza-ADM3union

 geo_name: ARG-Mendoza-ADM3

 geo_name: ARG-Mar_del_Plata-ADM3

 geo_name: ARG-Mar_del_Plata-ADM4

 geo_name: ARG-Ushuaia-ADM4

 geo_name: ARG-Ushuaia-ADM5

 geo_name: ARG-Salta-ADM2union

 geo_name: ARG-Salta-ADM3

 geo_name: ARG-Buenos_Aires-ADM2union

 geo_name: ARG-Buenos_Aires-ADM2

 geo_name: BRA-Teresina-ADM4union

 geo_name: BRA-Teresina-ADM4

 geo_name: BRA-Florianopolis-ADM4union

 geo_name: BRA-Florianopolis-ADM4

 geo_name: BRA-Belem-ADM4union

 geo_name: BRA-Belem-ADM4

 geo_name: CRI-San_Jose-ADM2union

 geo_name: CRI-San_Jose-ADM2

 geo_name: RWA-Kigali-ADM4union

 geo_name: RWA-Kigali-ADM4

 geo_name: SLE-Freetown-ADM4cityunion

 geo_name: SLE-Freetown-ADM4city

 geo_name: MAR-Marrakech-ADM2

 geo_name: MAR-Marrakech-ADM5

 geo_name: IND-Chennai-ADM4union

 geo_name: IND-Chennai-ADM4

 geo_name: IND-Pune-ADM4union

 geo_name: IND-Pune-ADM4

 geo_name: IND-Surat-ADM4union

 geo_name: IND-Surat-ADM4

 geo_name: CHN-Chengdu-ADM3union

 ge

In [16]:
processedcities = pd.read_csv(OUTPUT_FILENAME)
# ''out_dir +'/' OUTPUT_FILENAME'
processedcities

,Unnamed: 0,geo_id,geo_level,geo_name,geo_parent_name,exceedance_popfraction_2020,percent_of_WHO_2020
0,0,ARG-Mendoza_ADM-3-union_1,ADM-3-union,ARG-Mendoza,ARG-Mendoza,1.0,395.468104
1,0,ARG-Mendoza_ADM-3_1,ADM-3,Distrito Las Barrancas,ARG-Mendoza,1.0,421.848765
2,1,ARG-Mendoza_ADM-3_2,ADM-3,Distrito San Roque,ARG-Mendoza,1.0,418.691308
3,2,ARG-Mendoza_ADM-3_3,ADM-3,Distrito Fray Luis Beltrán,ARG-Mendoza,1.0,443.270436
4,3,ARG-Mendoza_ADM-3_4,ADM-3,Distrito Rodeo del Medio,ARG-Mendoza,1.0,435.658329
...,...,...,...,...,...,...,...
2623,82,RWA-Musanze_ADM5_83,ADM5,Kamajaga,RWA-Musanze,1.0,747.936003
2624,83,RWA-Musanze_ADM5_84,ADM5,Kamicaca,RWA-Musanze,1.0,753.422329
2625,84,RWA-Musanze_ADM5_85,ADM5,Musenyi,RWA-Musanze,1.0,754.165614
2626,85,RWA-Musanze_ADM5_86,ADM5,Rugari,RWA-Musanze,1.0,745.213966


# Merge with indicator table

In [20]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/indicators.csv')
cities_indicators

,geo_id,geo_level,geo_name,geo_parent_name,HEA_1_percentChangeinDaysAbove35C2020to2050,HEA_4_percentBuiltupWithoutTreeCover,HEA_2_percentBuiltupwHighLST-2013to2022meanofmonthwhottestday,ACC_2_percentOpenSpaceinBuiltup2022,ACC_1_OpenSpaceHectaresper1000people2022,FLD_2_percentChangeinMaxDailyPrecip2020to2050,FLD_1_percentFloodProneinBuiltup2050,FLD_4_percentImperviousinBuiltup2018,ACC_3_percentPopwOpenSpaceAccess2022,ACC_4_percentPopwTreeCoverAccess2022,FLD_3_percentBuiltupWithin1mAboveDrainage,FLD_7_percentSteepSlopesWOvegetationcover2020,FLD_6_percentRiparianZonewoVegorWatercover2020
0,ARG-Mendoza_ADM-3-union_1,ADM-3-union,ARG-Mendoza,ARG-Mendoza,0.0,NaN,0.306981,0.037594,350.814444,-0.041587,0.000098,0.010460,0.562167,NaN,NaN,0.991441,0.834118
1,ARG-Mendoza_ADM-3_1,ADM-3,Distrito Las Barrancas,ARG-Mendoza,NaN,NaN,0.317241,0.006540,1.972102,NaN,0.000000,0.000817,0.067684,NaN,NaN,0.500000,0.642378
2,ARG-Mendoza_ADM-3_2,ADM-3,Distrito San Roque,ARG-Mendoza,NaN,NaN,0.005444,0.006642,0.128004,NaN,0.000000,0.014849,0.214924,NaN,NaN,0.000000,0.448561
3,ARG-Mendoza_ADM-3_3,ADM-3,Distrito Fray Luis Beltrán,ARG-Mendoza,NaN,NaN,0.204788,0.007968,0.194904,NaN,0.000000,0.009535,0.172030,NaN,NaN,0.000000,0.504732
4,ARG-Mendoza_ADM-3_4,ADM-3,Distrito Rodeo del Medio,ARG-Mendoza,NaN,NaN,0.126250,0.005204,0.329989,NaN,0.000000,0.015890,0.084679,NaN,NaN,0.642857,0.399084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,RWA-Musanze_ADM5_83,ADM5,Kamajaga,RWA-Musanze,NaN,0.993718,0.078125,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.006873
2624,RWA-Musanze_ADM5_84,ADM5,Kamicaca,RWA-Musanze,NaN,0.995015,0.125000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
2625,RWA-Musanze_ADM5_85,ADM5,Musenyi,RWA-Musanze,NaN,0.983103,0.280000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.984223,0.013621,0.000000,0.039106
2626,RWA-Musanze_ADM5_86,ADM5,Rugari,RWA-Musanze,NaN,0.952790,0.000000,0.000000,0.000000,NaN,0.000000,0.014102,0.000000,1.000000,0.001839,0.000000,0.000000


In [21]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    else:
        print("add new indicators")
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    return(cities_indicators_df)

In [22]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = processedcities,
                                            indicator_name = 'AQ_3_percentWHOlimitPM25exposure2020')

add new indicators


In [23]:
cities_indicators_merged

,geo_id,geo_level,geo_name,geo_parent_name,HEA_1_percentChangeinDaysAbove35C2020to2050,HEA_4_percentBuiltupWithoutTreeCover,HEA_2_percentBuiltupwHighLST-2013to2022meanofmonthwhottestday,ACC_2_percentOpenSpaceinBuiltup2022,ACC_1_OpenSpaceHectaresper1000people2022,FLD_2_percentChangeinMaxDailyPrecip2020to2050,FLD_1_percentFloodProneinBuiltup2050,FLD_4_percentImperviousinBuiltup2018,ACC_3_percentPopwOpenSpaceAccess2022,ACC_4_percentPopwTreeCoverAccess2022,FLD_3_percentBuiltupWithin1mAboveDrainage,FLD_7_percentSteepSlopesWOvegetationcover2020,FLD_6_percentRiparianZonewoVegorWatercover2020,AQ_3_percentWHOlimitPM25exposure2020
0,ARG-Mendoza_ADM-3-union_1,ADM-3-union,ARG-Mendoza,ARG-Mendoza,0.0,NaN,0.306981,0.037594,350.814444,-0.041587,0.000098,0.010460,0.562167,NaN,NaN,0.991441,0.834118,395.468104
1,ARG-Mendoza_ADM-3_1,ADM-3,Distrito Las Barrancas,ARG-Mendoza,NaN,NaN,0.317241,0.006540,1.972102,NaN,0.000000,0.000817,0.067684,NaN,NaN,0.500000,0.642378,421.848765
2,ARG-Mendoza_ADM-3_2,ADM-3,Distrito San Roque,ARG-Mendoza,NaN,NaN,0.005444,0.006642,0.128004,NaN,0.000000,0.014849,0.214924,NaN,NaN,0.000000,0.448561,418.691308
3,ARG-Mendoza_ADM-3_3,ADM-3,Distrito Fray Luis Beltrán,ARG-Mendoza,NaN,NaN,0.204788,0.007968,0.194904,NaN,0.000000,0.009535,0.172030,NaN,NaN,0.000000,0.504732,443.270436
4,ARG-Mendoza_ADM-3_4,ADM-3,Distrito Rodeo del Medio,ARG-Mendoza,NaN,NaN,0.126250,0.005204,0.329989,NaN,0.000000,0.015890,0.084679,NaN,NaN,0.642857,0.399084,435.658329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,RWA-Musanze_ADM5_83,ADM5,Kamajaga,RWA-Musanze,NaN,0.993718,0.078125,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.006873,747.936003
2624,RWA-Musanze_ADM5_84,ADM5,Kamicaca,RWA-Musanze,NaN,0.995015,0.125000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,753.422329
2625,RWA-Musanze_ADM5_85,ADM5,Musenyi,RWA-Musanze,NaN,0.983103,0.280000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.984223,0.013621,0.000000,0.039106,754.165614
2626,RWA-Musanze_ADM5_86,ADM5,Rugari,RWA-Musanze,NaN,0.952790,0.000000,0.000000,0.000000,NaN,0.000000,0.014102,0.000000,1.000000,0.001839,0.000000,0.000000,745.213966


# Upload in aws s3

In [24]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
# aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [25]:
# upload to aws
key_data = 'indicators/indicators.csv'

cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [26]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')